In [2]:
import tensorflow as tf
import os
from tensorflow import keras
import numpy as np
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from multiprocessing import Process
from IPython.display import clear_output

2024-06-01 16:22:11.465766: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-01 16:22:11.491373: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-01 16:22:11.634332: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 16:22:13.035904: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
FRAME_SKIP = 2
FRAME_SIZE = (150,150)

In [4]:
# !rm -r A-Dataset-for-Automatic-Violence-Detection-in-Videos/
# !git clone https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos

In [5]:
# !rm -r Data
# !mkdir Data
# !mkdir -p ./Data/Video/Violent
# !mkdir -p ./Data/Video/NonViolent
# !cp -a ./A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/violent/cam1/. ./Data/Video/Violent/
# !cp -a ./A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/non-violent/cam1/. ./Data/Video/NonViolent/
# clear_output()
# !mkdir -p ./Data/Training/V
# !mkdir -p ./Data/Training/NV

In [6]:
def extract_frames(video_path, output_path, frame_size=(150, 150), frame_skip=2):
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_skip == 0:
            frame = cv2.resize(frame, frame_size)
            frame_filename = os.path.join(output_path, f"{os.path.splitext(os.path.basename(video_path))[0]}_frame_{count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
        count += 1
    cap.release()

def thread_1():
    for i in range(60):
        video_path = f"./Data/Video/Violent/{i+1}.mp4"
        output_path = "./Data/Training/V"
        print(f"Processing Violent Vid-{i}")
        extract_frames(video_path, output_path, frame_size=FRAME_SIZE, frame_skip=FRAME_SKIP)
    print("Violent Extracted")

def thread_2():
    for i in range(60):
        video_path = f"./Data/Video/NonViolent/{i+1}.mp4"
        output_path = "./Data/Training/NV"
        print(f"Processing NonViolent Vid-{i}")
        extract_frames(video_path, output_path, frame_size=FRAME_SIZE, frame_skip=FRAME_SKIP)
    print("Non-Violent Extracted")

In [7]:
t1 = Process(target=thread_1, args=())
t2 = Process(target=thread_2, args=())

t1.start() 
t2.start()

t1.join()
t2.join()
print("Complete")

Processing Violent Vid-0
Processing NonViolent Vid-0


Processing Violent Vid-1
Processing NonViolent Vid-1
Processing NonViolent Vid-2
Processing Violent Vid-2
Processing NonViolent Vid-3
Processing Violent Vid-3
Processing NonViolent Vid-4
Processing Violent Vid-4
Processing Violent Vid-5
Processing NonViolent Vid-5
Processing Violent Vid-6
Processing NonViolent Vid-6
Processing Violent Vid-7
Processing NonViolent Vid-7
Processing NonViolent Vid-8
Processing Violent Vid-8
Processing NonViolent Vid-9
Processing Violent Vid-9
Processing NonViolent Vid-10
Processing Violent Vid-10
Processing NonViolent Vid-11
Processing Violent Vid-11
Processing NonViolent Vid-12
Processing Violent Vid-12
Processing Violent Vid-13
Processing NonViolent Vid-13
Processing Violent Vid-14
Processing NonViolent Vid-14
Processing Violent Vid-15
Processing NonViolent Vid-15
Processing Violent Vid-16
Processing NonViolent Vid-16
Processing Violent Vid-17
Processing Violent Vid-18
Processing NonViolent Vid-17
Processing Violent Vid-19
Processing NonViolent Vid-18
Pr

In [8]:
base_dir='./Data'
train_dir=os.path.join(base_dir,'Training')
train_violent_dir =os.path.join(train_dir, 'V' )
train_nonviolent_dir=os.path.join(train_dir,'NV')

In [9]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=40,width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,horizontal_flip=True, fill_mode='nearest')

In [10]:
train_generator = train_datagen.flow_from_directory(train_dir,color_mode="rgb", target_size = FRAME_SIZE,batch_size=20,classes=['NV','V'], class_mode='binary', shuffle=True)

Found 10280 images belonging to 2 classes.


In [11]:
model= tf.keras.models.Sequential([
       tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Dropout(0.5),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(512, activation='relu'),
       tf.keras.layers.Dense(1,activation ='sigmoid')

])

/home/shobhits/Documents/Mini Project/CrimeDetection/env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
model1=model.fit(train_generator,steps_per_epoch=50, epochs=30)

Epoch 1/30


/home/shobhits/Documents/Mini Project/CrimeDetection/env/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 40s 740ms/step - accuracy: 0.5098 - loss: 0.7030
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 716ms/step - accuracy: 0.6049 - loss: 0.6745
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 700ms/step - accuracy: 0.6936 - loss: 0.5739
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 711ms/step - accuracy: 0.7612 - loss: 0.5106
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 723ms/step - accuracy: 0.7687 - loss: 0.4867
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 717ms/step - accuracy: 0.7428 - loss: 0.5159
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 677ms/step - accuracy: 0.8172 - loss: 0.4275
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 37s 737ms/step - accuracy: 0.7949 - loss: 0.4191
Epoch 9/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 711ms/step - accuracy: 0.8222 - loss: 0.4185
Epoch 10/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 680ms/step - accuracy: 0.8364 - loss: 0.3956
Epoch 11/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 183ms/step - accuracy: 0.8124 - loss: 0.4029
Epoch 12/30


2024-06-01 16:30:18.188967: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 822ms/step - accuracy: 0.8513 - loss: 0.3553
Epoch 13/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 706ms/step - accuracy: 0.8535 - loss: 0.3564
Epoch 14/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 704ms/step - accuracy: 0.8215 - loss: 0.4301
Epoch 15/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 680ms/step - accuracy: 0.8589 - loss: 0.3399
Epoch 16/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 696ms/step - accuracy: 0.8534 - loss: 0.3453
Epoch 17/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 701ms/step - accuracy: 0.8765 - loss: 0.3090
Epoch 18/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 692ms/step - accuracy: 0.8125 - loss: 0.4038
Epoch 19/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 693ms/step - accuracy: 0.8480 - loss: 0.3337
Epoch 20/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 705ms/step - accuracy: 0.8597 - loss: 0.3387
Epoch 21/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 680ms/step - accuracy: 0.8718 - loss: 0.3189
Epoch 22/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 177ms/step - accuracy: 0.7874 - loss: 0.4381 
Epoch 23/30


2024-06-01 16:36:22.410893: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 730ms/step - accuracy: 0.8245 - loss: 0.3795
Epoch 24/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 696ms/step - accuracy: 0.8651 - loss: 0.3325
Epoch 25/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 704ms/step - accuracy: 0.8627 - loss: 0.3183
Epoch 26/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 691ms/step - accuracy: 0.8692 - loss: 0.3165
Epoch 27/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 700ms/step - accuracy: 0.8336 - loss: 0.3335
Epoch 28/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 712ms/step - accuracy: 0.8780 - loss: 0.3025
Epoch 29/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 725ms/step - accuracy: 0.8845 - loss: 0.3041
Epoch 30/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 37s 732ms/step - accuracy: 0.8881 - loss: 0.2614


In [14]:
import time
t = time.time()
export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

./1717240268.h5
